# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [2]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [4]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

# 1. Drop high-cardinality/unnecessary columns
df = spaceship.drop(['PassengerId', 'Name', 'Cabin'], axis=1)

# 2. Handle missing values
# Numerical: Fill with median
num_cols = df.select_dtypes(include=['float64']).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Categorical: Fill with mode
cat_cols = df.select_dtypes(include=['object', 'bool']).columns
for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

# 3. Convert Boolean 'Transported' to numeric (0, 1)
df['Transported'] = df['Transported'].astype(int)

# 4. One-Hot Encoding for categorical features
df = pd.get_dummies(df, columns=['HomePlanet', 'CryoSleep', 'Destination', 'VIP'], drop_first=True)

# 5. Feature Scaling
scaler = StandardScaler()
X = df.drop('Transported', axis=1)
y = df['Transported']
X_scaled = scaler.fit_transform(X)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_29676\476204874.py:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(df[col].mode()[0])


**Perform Train Test Split**

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [6]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# Bagging (bootstrap=True)
bagging = BaggingClassifier(DecisionTreeClassifier(), n_estimators=100, bootstrap=True, random_state=42)
bagging.fit(X_train, y_train)
print(f"Bagging Score: {bagging.score(X_test, y_test):.4f}")

Bagging Score: 0.7740


- Random Forests

In [7]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
print(f"Random Forest Score: {rf.score(X_test, y_test):.4f}")

Random Forest Score: 0.7792


- Gradient Boosting

In [8]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
gb.fit(X_train, y_train)
print(f"Gradient Boosting Score: {gb.score(X_test, y_test):.4f}")

Gradient Boosting Score: 0.7838


- Adaptive Boosting

In [9]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier(n_estimators=100, random_state=42)
ada.fit(X_train, y_train)
print(f"AdaBoost Score: {ada.score(X_test, y_test):.4f}")

AdaBoost Score: 0.7711


Which model is the best and why?

In [ ]:
'''
The winner is Gradient Boosting! --fireworks sounds--
Here is why I think it worked best
The Score: It just got the highest number (0.78) compared to all the others I ran
How it learns: Gradient Boosting is pretty smart because it builds trees one after another instead of just doing a bunch at once, it looks at the mistakes the last tree made and tries to fix them in the next one.
Better Tuning: It uses gradient descent to lower the errors, which usually makes it really good for this kind of data table.
'''